In [1]:
from datasets import CIFAR10_truncated
import argparse
from models import *
from utils import progress_bar

from torchvision import transforms
import torch.utils.data as data
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import os
import matplotlib.pyplot as plt
import numpy as np


def get_dataloader(datadir, train_bs, test_bs, dataidxs=None):
    transform = transforms.Compose([transforms.ToTensor()])
    train_ds = CIFAR10_truncated(datadir, dataidxs=dataidxs, train=True, transform=transform, download=True)
    test_ds = CIFAR10_truncated(datadir, train=False, transform=transform, download=True)
    train_dl = data.DataLoader(dataset=train_ds, batch_size=train_bs, shuffle=True, num_workers=2)
    test_dl = data.DataLoader(dataset=test_ds, batch_size=test_bs, shuffle=False, num_workers=2)
    return train_dl, test_dl


def adjust_learning_rate(optimizer, epoch, lr):
    if epoch in args.lr_schedule:
        lr *= args.lr_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr



dataidxs = []
# load the index of imbalanced CIFAR-10 from dataidx.txt
with open("dataidx.txt", "r") as f:
    for line in f:
        dataidxs.append(int(line.strip()))
# get the training/testing data loader
train_dl, test_dl = get_dataloader("./data/cifar10", 256, 100, dataidxs)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_dl.dataset.target

(23654,)

In [46]:
def countX(tup, x): 
    count = 0
    for ele in tup: 
        if (ele == x): 
            count = count + 1
    return count 

target=train_dl.dataset.target

dicts_target = {}
beta=0.999
weights=[]

for a in range (0,10):
    dicts_target[a]=countX(target,a)
    
for key,value in dicts_target.items():
    weight=(1-beta)/(1-(beta**value))
    weights.append(weight)
    

In [47]:
target

array([4, 6, 4, ..., 5, 4, 2])

In [53]:
sum(dicts_target.values())

23654

In [71]:
train_ds = CIFAR10_truncated("./data/cifar10", 256, 100, 90)


In [76]:
    train_ds.targets

AttributeError: 'CIFAR10_truncated' object has no attribute 'targets'

In [73]:
dataidxs

[20137,
 12321,
 20306,
 5226,
 32305,
 18092,
 46528,
 5148,
 33080,
 12335,
 38498,
 48989,
 40642,
 37860,
 43118,
 32650,
 15461,
 47932,
 27613,
 7644,
 41071,
 22107,
 12394,
 17048,
 8833,
 19859,
 19154,
 41682,
 15667,
 46448,
 18459,
 44349,
 30198,
 11091,
 37568,
 38236,
 3125,
 6090,
 10834,
 11266,
 3508,
 18409,
 48398,
 42249,
 42216,
 20260,
 36289,
 3963,
 30003,
 49967,
 27776,
 43575,
 8350,
 11769,
 33063,
 4733,
 8947,
 30293,
 41442,
 32207,
 9520,
 31968,
 43221,
 39478,
 6144,
 41017,
 22908,
 38164,
 28696,
 18064,
 11000,
 3155,
 4695,
 16666,
 959,
 15431,
 3397,
 12206,
 26804,
 36453,
 24943,
 14010,
 17185,
 19010,
 29610,
 41999,
 28755,
 38123,
 23026,
 16805,
 11185,
 33424,
 3359,
 33945,
 22469,
 18723,
 32578,
 36710,
 14340,
 13173,
 38660,
 35602,
 7649,
 1856,
 20107,
 32016,
 26349,
 15910,
 17165,
 32187,
 18827,
 13985,
 12989,
 38140,
 18504,
 32419,
 41263,
 42449,
 22085,
 40336,
 40951,
 1403,
 21915,
 20659,
 43296,
 28234,
 15458,
 29694